In [2]:
#!/usr/bin/env python
#-*- coding:utf-8 -*-
import re,io
from urllib import request
import time
import sys,os
import xlrd
import csv
file = xlrd.open_workbook("/Users/apple/Desktop/IGEM/post/neuron network/AWANeuronsFlow/NeuronType.xls")
sheet = file.sheet_by_index(0)
colnums = sheet.ncols
Neurons = sheet.col_values(0)
del Neurons[0]
ReduNeurons = []
for ne in Neurons:
    ReduNeurons.append(ne)
    ReduNeurons.append(ne[0:-1])
    ReduNeurons.append(ne[0:-2])

In [3]:
print(len(Neurons))

279


In [4]:
print(Neurons)

['ADAL', 'ADAR', 'ADEL', 'ADER', 'ADFL', 'ADFR', 'ADLL', 'ADLR', 'AFDL', 'AFDR', 'AIAL', 'AIAR', 'AIBL', 'AIBR', 'AIML', 'AIMR', 'AINL', 'AINR', 'AIYL', 'AIYR', 'AIZL', 'AIZR', 'ALA', 'ALML', 'ALMR', 'ALNL', 'ALNR', 'AQR', 'AS01', 'AS02', 'AS03', 'AS04', 'AS05', 'AS06', 'AS07', 'AS08', 'AS09', 'AS10', 'AS11', 'ASEL', 'ASER', 'ASGL', 'ASGR', 'ASHL', 'ASHR', 'ASIL', 'ASIR', 'ASJL', 'ASJR', 'ASKL', 'ASKR', 'AUAL', 'AUAR', 'AVAL', 'AVAR', 'AVBL', 'AVBR', 'AVDL', 'AVDR', 'AVEL', 'AVER', 'AVFL', 'AVFR', 'AVG', 'AVHL', 'AVHR', 'AVJL', 'AVJR', 'AVKL', 'AVKR', 'AVL', 'AVM', 'AWAL', 'AWAR', 'AWBL', 'AWBR', 'AWCL', 'AWCR', 'BAGL', 'BAGR', 'BDUL', 'BDUR', 'CEPDL', 'CEPDR', 'CEPVL', 'CEPVR', 'DA01', 'DA02', 'DA03', 'DA04', 'DA05', 'DA06', 'DA07', 'DA08', 'DA09', 'DB01', 'DB02', 'DB03', 'DB04', 'DB05', 'DB06', 'DB07', 'DD01', 'DD02', 'DD03', 'DD04', 'DD05', 'DD06', 'DVA', 'DVB', 'DVC', 'FLPL', 'FLPR', 'HSNL', 'HSNR', 'IL1DL', 'IL1DR', 'IL1L', 'IL1R', 'IL1VL', 'IL1VR', 'IL2DL', 'IL2DR', 'IL2L', 'IL2R

In [5]:
print(ReduNeurons)

['ADAL', 'ADA', 'AD', 'ADAR', 'ADA', 'AD', 'ADEL', 'ADE', 'AD', 'ADER', 'ADE', 'AD', 'ADFL', 'ADF', 'AD', 'ADFR', 'ADF', 'AD', 'ADLL', 'ADL', 'AD', 'ADLR', 'ADL', 'AD', 'AFDL', 'AFD', 'AF', 'AFDR', 'AFD', 'AF', 'AIAL', 'AIA', 'AI', 'AIAR', 'AIA', 'AI', 'AIBL', 'AIB', 'AI', 'AIBR', 'AIB', 'AI', 'AIML', 'AIM', 'AI', 'AIMR', 'AIM', 'AI', 'AINL', 'AIN', 'AI', 'AINR', 'AIN', 'AI', 'AIYL', 'AIY', 'AI', 'AIYR', 'AIY', 'AI', 'AIZL', 'AIZ', 'AI', 'AIZR', 'AIZ', 'AI', 'ALA', 'AL', 'A', 'ALML', 'ALM', 'AL', 'ALMR', 'ALM', 'AL', 'ALNL', 'ALN', 'AL', 'ALNR', 'ALN', 'AL', 'AQR', 'AQ', 'A', 'AS01', 'AS0', 'AS', 'AS02', 'AS0', 'AS', 'AS03', 'AS0', 'AS', 'AS04', 'AS0', 'AS', 'AS05', 'AS0', 'AS', 'AS06', 'AS0', 'AS', 'AS07', 'AS0', 'AS', 'AS08', 'AS0', 'AS', 'AS09', 'AS0', 'AS', 'AS10', 'AS1', 'AS', 'AS11', 'AS1', 'AS', 'ASEL', 'ASE', 'AS', 'ASER', 'ASE', 'AS', 'ASGL', 'ASG', 'AS', 'ASGR', 'ASG', 'AS', 'ASHL', 'ASH', 'AS', 'ASHR', 'ASH', 'AS', 'ASIL', 'ASI', 'AS', 'ASIR', 'ASI', 'AS', 'ASJL', 'ASJ', 'AS

In [6]:
def FindNeuronPromoters(neuron):
    #http://www.wormweb.org/neuralnet#c=AS&m=1
    #http://www.wormweb.org/fetchData2.php?c=AIA&m=1&t=all
    # http://www.wormweb.org/fetchData2.php?c=RIVL&m=0&t=all
    searchUrl = "http://www.wormweb.org/fetchData2.php?c=" + neuron + "&m=1&t=all"
    response = str(request.urlopen(searchUrl).read(),"utf-8")
    #print(response)
    #searchResult = re.search(r"(genes</a> express in <b>.*gene pairs</a> uniquely overlap in)",str(response))
    #searchResult = re.search(r"(?s)<span class=\"keyword\">.*?<div class=\"trans-container\">.*?<ul>.*?</div>",str(response))
    GeneExp, UniqExp = None, None
    GeneExp = re.search(r'gene(s)?</a> express(es)? in <b>.* uniquely express(es)? in',response)
    if GeneExp:
        GeneExp = re.findall(r'<i>([a-z]{3}-[0-9]{1,2})</i> ',GeneExp.group())
    UniqExp = re.search(r'gene(s)?</a> uniquely express(es)? in.*uniquely overlap(s)? in',response)
    if UniqExp:
        UniqExp = re.findall(r'<i>([a-z]{3}-[0-9]{1,2})</i> ',UniqExp.group())
    print(GeneExp)
    print(UniqExp)
    #UniqExp = re.findall(r'gene pairs</a> uniquely overlap in.*physiology</a> data for',response)
    return GeneExp,UniqExp

In [7]:
GeneExp,UniqExp = FindNeuronPromoters('OLQ')
print(GeneExp,UniqExp)

['che-3', 'del-2', 'dyf-11', 'eat-20', 'eat-4', 'npr-1', 'ocr-4', 'osm-6', 'osm-9', 'sox-3']
['ocr-4']
['che-3', 'del-2', 'dyf-11', 'eat-20', 'eat-4', 'npr-1', 'ocr-4', 'osm-6', 'osm-9', 'sox-3'] ['ocr-4']


In [8]:
GeneExp,UniqExp =  FindNeuronPromoters('ADA')

if GeneExp is None and UniqExp is None:
    print('empty',neu)
    pass
else: 
    print('123')

['eat-4', 'flp-8', 'inx-4', 'nsy-5', 'pdf-1', 'unc-86', 'unc-9']
None
123


In [9]:
#flags Gene Expression = 'GeneExp'', Uniquely expression = 'UniqExp'
NeuronPromoters = {}
count = 0
for neu in ReduNeurons:
    print()
    print(neu)
    try:
        GeneExp,UniqExp =  FindNeuronPromoters(neu)
        if GeneExp is None and UniqExp is None:
            print('empty',neu)
            pass
        else: 
            NeuronPromoters[neu] = {}
            NeuronPromoters[neu]['GeneExp'] = GeneExp
            NeuronPromoters[neu]['UniqExp'] = UniqExp
            count = count+1
            #print('Y  '+neu)
    except:
        print('No '+neu)
        pass
    count  = count+1
    #if count ==20:
       #break
print(count)


ADAL
None
None
empty ADAL

ADA
['eat-4', 'flp-8', 'inx-4', 'nsy-5', 'pdf-1', 'unc-86', 'unc-9']
None

AD
None
None
empty AD

ADAR
None
None
empty ADAR

ADA
['eat-4', 'flp-8', 'inx-4', 'nsy-5', 'pdf-1', 'unc-86', 'unc-9']
None

AD
None
None
empty AD

ADEL
None
None
empty ADEL

ADE
['acr-16', 'ast-1', 'bas-1', 'cat-1', 'cat-2', 'cat-4', 'che-2', 'che-3', 'dat-1', 'dop-2', 'dyf-11', 'exp-1', 'gpa-14', 'inx-4', 'inx-7', 'osm-6', 'pdf-1', 'snf-3', 'tax-2', 'trp-4', 'unc-9']
None

AD
None
None
empty AD

ADER
None
None
empty ADER

ADE
['acr-16', 'ast-1', 'bas-1', 'cat-1', 'cat-2', 'cat-4', 'che-2', 'che-3', 'dat-1', 'dop-2', 'dyf-11', 'exp-1', 'gpa-14', 'inx-4', 'inx-7', 'osm-6', 'pdf-1', 'snf-3', 'tax-2', 'trp-4', 'unc-9']
None

AD
None
None
empty AD

ADFL
None
None
empty ADFL

ADF
['bas-1', 'cat-1', 'che-12', 'che-2', 'che-3', 'dyf-11', 'eat-4', 'egl-30', 'flp-21', 'glb-5', 'gpa-10', 'gpa-13', 'gpa-3', 'ins-1', 'ins-7', 'inx-4', 'kvs-1', 'lin-11', 'mgl-3', 'mps-1', 'ncs-1', 'nlp-3', 'npr-5

None
None
empty AS02

AS0
None
None
empty AS0

AS
['ace-2', 'acr-14', 'acr-15', 'cha-1', 'cho-1', 'ckr-2', 'cnd-1', 'dgk-1', 'dop-1', 'dop-3', 'eat-6', 'gar-2', 'gar-3', 'gbb-1', 'goa-1', 'inx-3', 'kvs-1', 'max-1', 'nlg-1', 'nlp-21', 'rab-3', 'ric-7', 'rig-3', 'slo-2', 'tig-2', 'tsp-7', 'twk-13', 'twk-40', 'twk-7', 'unc-17', 'unc-3', 'unc-40', 'unc-57', 'unc-63', 'unc-7', 'unc-8', 'vab-3']
None

AS03
None
None
empty AS03

AS0
None
None
empty AS0

AS
['ace-2', 'acr-14', 'acr-15', 'cha-1', 'cho-1', 'ckr-2', 'cnd-1', 'dgk-1', 'dop-1', 'dop-3', 'eat-6', 'gar-2', 'gar-3', 'gbb-1', 'goa-1', 'inx-3', 'kvs-1', 'max-1', 'nlg-1', 'nlp-21', 'rab-3', 'ric-7', 'rig-3', 'slo-2', 'tig-2', 'tsp-7', 'twk-13', 'twk-40', 'twk-7', 'unc-17', 'unc-3', 'unc-40', 'unc-57', 'unc-63', 'unc-7', 'unc-8', 'vab-3']
None

AS04
None
None
empty AS04

AS0
None
None
empty AS0

AS
['ace-2', 'acr-14', 'acr-15', 'cha-1', 'cho-1', 'ckr-2', 'cnd-1', 'dgk-1', 'dop-1', 'dop-3', 'eat-6', 'gar-2', 'gar-3', 'gbb-1', 'goa-1', 'inx

None
None
empty ASHR

ASH
['che-12', 'che-2', 'che-3', 'cng-1', 'del-2', 'dyf-11', 'eat-16', 'eat-4', 'egl-3', 'flp-21', 'gpa-1', 'gpa-11', 'gpa-13', 'gpa-14', 'gpa-15', 'gpa-3', 'gpa-6', 'gpc-1', 'ins-1', 'inx-4', 'kvs-1', 'mps-1', 'nlp-15', 'nlp-3', 'npr-1', 'nsy-5', 'ntr-1', 'ocr-2', 'odr-3', 'odr-4', 'osm-3', 'osm-6', 'osm-9', 'qui-1', 'rgs-3', 'ser-5', 'sra-6', 'srb-6', 'tax-6', 'tmc-1', 'ttx-4', 'ugt-56', 'unc-42', 'unc-8']
None

AS
['ace-2', 'acr-14', 'acr-15', 'cha-1', 'cho-1', 'ckr-2', 'cnd-1', 'dgk-1', 'dop-1', 'dop-3', 'eat-6', 'gar-2', 'gar-3', 'gbb-1', 'goa-1', 'inx-3', 'kvs-1', 'max-1', 'nlg-1', 'nlp-21', 'rab-3', 'ric-7', 'rig-3', 'slo-2', 'tig-2', 'tsp-7', 'twk-13', 'twk-40', 'twk-7', 'unc-17', 'unc-3', 'unc-40', 'unc-57', 'unc-63', 'unc-7', 'unc-8', 'vab-3']
None

ASIL
None
None
empty ASIL

ASI
['che-12', 'che-2', 'che-3', 'cng-1', 'cng-2', 'cng-3', 'daf-11', 'daf-28', 'daf-37', 'daf-38', 'daf-7', 'dyf-11', 'flp-10', 'flp-2', 'flp-21', 'gcy-1', 'gcy-2', 'gcy-27', 'gcy-

None
None
empty AV

AVDR
None
None
empty AVDR

AVD
['aex-2', 'cfi-1', 'egl-3', 'glr-1', 'glr-2', 'glr-5', 'gur-3', 'inx-6', 'nmr-1', 'nmr-2', 'pdf-2', 'rig-1', 'trp-2', 'unc-8', 'unc-9']
None

AV
None
None
empty AV

AVEL
None
None
empty AVEL

AVE
['eat-4', 'flp-1', 'glr-1', 'glr-2', 'glr-5', 'ncs-1', 'nmr-1', 'nmr-2', 'opt-3', 'rig-1', 'tax-6', 'trp-2', 'unc-7', 'wrk-1']
None

AV
None
None
empty AV

AVER
None
None
empty AVER

AVE
['eat-4', 'flp-1', 'glr-1', 'glr-2', 'glr-5', 'ncs-1', 'nmr-1', 'nmr-2', 'opt-3', 'rig-1', 'tax-6', 'trp-2', 'unc-7', 'wrk-1']
None

AV
None
None
empty AV

AVFL
None
None
empty AVFL

AVF
['inx-8', 'mec-7', 'pag-3', 'unc-4']
None

AV
None
None
empty AV

AVFR
None
None
empty AVFR

AVF
['inx-8', 'mec-7', 'pag-3', 'unc-4']
None

AV
None
None
empty AV

AVG
['deg-3', 'flp-22', 'flp-7', 'glr-1', 'glr-2', 'ins-17', 'inx-18', 'lin-11', 'nmr-1', 'nmr-2', 'ntr-2', 'odr-2', 'pdf-2', 'unc-6', 'unc-7']
['ins-17']

AV
None
None
empty AV

A
None
None
empty A

AVHL
None
None
e

None
None
empty DA03

DA0
None
None
empty DA0

DA
['acc-4', 'ace-2', 'acr-12', 'acr-14', 'acr-15', 'acr-2', 'avr-15', 'cha-1', 'cho-1', 'ckr-2', 'cnd-1', 'dbl-1', 'dgk-1', 'dop-1', 'dop-3', 'eat-6', 'flp-11', 'gar-2', 'gar-3', 'gbb-1', 'goa-1', 'inx-1', 'inx-3', 'inx-7', 'itr-1', 'kvs-1', 'max-1', 'mbl-1', 'mig-13', 'nca-1', 'nlg-1', 'nlp-21', 'pkc-1', 'rab-3', 'ric-7', 'rig-3', 'rig-4', 'slo-2', 'sox-2', 'syg-1', 'syg-2', 'tig-2', 'trp-1', 'tsp-7', 'twk-13', 'twk-40', 'twk-43', 'twk-7', 'unc-17', 'unc-3', 'unc-31', 'unc-37', 'unc-4', 'unc-40', 'unc-53', 'unc-57', 'unc-63', 'unc-7', 'unc-8', 'unc-9', 'wrk-1']
['mig-13']

DA04
None
None
empty DA04

DA0
None
None
empty DA0

DA
['acc-4', 'ace-2', 'acr-12', 'acr-14', 'acr-15', 'acr-2', 'avr-15', 'cha-1', 'cho-1', 'ckr-2', 'cnd-1', 'dbl-1', 'dgk-1', 'dop-1', 'dop-3', 'eat-6', 'flp-11', 'gar-2', 'gar-3', 'gbb-1', 'goa-1', 'inx-1', 'inx-3', 'inx-7', 'itr-1', 'kvs-1', 'max-1', 'mbl-1', 'mig-13', 'nca-1', 'nlg-1', 'nlp-21', 'pkc-1', 'rab-3', 'r

...

['cfi-1', 'eat-4', 'flp-11', 'flp-20', 'glr-5', 'gpa-10', 'inx-11', 'inx-13', 'inx-7', 'itr-1', 'nlp-13', 'pdf-1', 'unc-9']
None

LU
None
None
empty LU

LUAR
None
None
empty LUAR

LUA
['cfi-1', 'eat-4', 'flp-11', 'flp-20', 'glr-5', 'gpa-10', 'inx-11', 'inx-13', 'inx-7', 'itr-1', 'nlp-13', 'pdf-1', 'unc-9']
None

LU
None
None
empty LU

OLLL
None
None
empty OLLL

OLL
['ace-1', 'che-2', 'che-3', 'dop-6', 'dyf-11', 'eat-4', 'egl-3', 'flp-3', 'gpa-2', 'grd-8', 'inx-19', 'osm-6', 'pqn-47', 'ric-4', 'ser-2', 'sox-2', 'uig-1', 'unc-9']
['dop-6', 'grd-8', 'pqn-47', 'uig-1']

OL
None
None
empty OL

OLLR
None
None
empty OLLR

OLL
['ace-1', 'che-2', 'che-3', 'dop-6', 'dyf-11', 'eat-4', 'egl-3', 'flp-3', 'gpa-2', 'grd-8', 'inx-19', 'osm-6', 'pqn-47', 'ric-4', 'ser-2', 'sox-2', 'uig-1', 'unc-9']
['dop-6', 'grd-8', 'pqn-47', 'uig-1']

OL
None
None
empty OL

OLQDL
None
None
empty OLQDL

OLQD
None
None
empty OLQD

OLQ
['che-3', 'del-2', 'dyf-11', 'eat-20', 'eat-4', 'npr-1', 'ocr-4', 'osm-6', 'osm-9', '

None
None
empty PV

PVR
['ceh-14', 'eat-4', 'flp-10', 'flp-12', 'flp-20', 'ggr-1', 'inx-13', 'inx-3', 'inx-4', 'inx-8', 'inx-9', 'ntr-1', 'ntr-2', 'unc-86', 'unc-9']
None

PV
None
None
empty PV

P
None
None
empty P

PVT
['ceh-14', 'flp-20', 'flp-5', 'flp-6', 'gcy-1', 'gcy-14', 'gcy-2', 'gcy-3', 'gpa-2', 'gpa-3', 'inx-17', 'inx-18', 'inx-3', 'inx-4', 'inx-7', 'lim-6', 'pdf-1', 'pdf-2', 'ser-1', 'ser-4', 'sra-11', 'syg-2', 'trp-2', 'unc-6', 'unc-9', 'zag-1', 'zig-2']
['zig-2']

PV
None
None
empty PV

P
None
None
empty P

PVWL
None
None
empty PVWL

PVW
['ceh-14', 'flp-2', 'flp-22', 'flp-7', 'inx-7', 'ntr-1', 'ntr-2', 'unc-9']
None

PV
None
None
empty PV

PVWR
None
None
empty PVWR

PVW
['ceh-14', 'flp-2', 'flp-22', 'flp-7', 'inx-7', 'ntr-1', 'ntr-2', 'unc-9']
None

PV
None
None
empty PV

RIAL
None
None
empty RIAL

RIA
['daf-16', 'dop-2', 'eat-4', 'gcy-2', 'glr-1', 'glr-2', 'glr-3', 'glr-6', 'glt-4', 'gpa-14', 'ins-3', 'ins-4', 'nlp-22', 'rig-4', 'ser-1', 'ser-2', 'tax-6', 'trp-1', 'unc-7']

None
None
empty SA

SABVL
None
None
empty SABVL

SABV
None
None
empty SABV

SAB
['ceh-6', 'cho-1', 'del-1', 'exp-1', 'flp-11', 'glr-4', 'glr-5', 'inx-8', 'ric-19', 'ric-4', 'snb-1', 'syg-1', 'unc-10', 'unc-17', 'unc-3', 'unc-31', 'unc-4', 'unc-8']
None

SABVR
None
None
empty SABVR

SABV
None
None
empty SABV

SAB
['ceh-6', 'cho-1', 'del-1', 'exp-1', 'flp-11', 'glr-4', 'glr-5', 'inx-8', 'ric-19', 'ric-4', 'snb-1', 'syg-1', 'unc-10', 'unc-17', 'unc-3', 'unc-31', 'unc-4', 'unc-8']
None

SDQL
None
None
empty SDQL

SDQ
['egl-3', 'egl-6', 'flp-12', 'gcy-35', 'inx-7', 'lgc-55', 'npr-1', 'unc-2', 'unc-31', 'unc-36', 'unc-86', 'unc-9']
None

SD
None
None
empty SD

SDQR
None
None
empty SDQR

SDQ
['egl-3', 'egl-6', 'flp-12', 'gcy-35', 'inx-7', 'lgc-55', 'npr-1', 'unc-2', 'unc-31', 'unc-36', 'unc-86', 'unc-9']
None

SD
None
None
empty SD

SIADL
None
None
empty SIADL

SIAD
None
None
empty SIAD

SIA
['acr-15', 'dop-2', 'dop-3', 'ggr-2', 'ggr-3', 'odr-2', 'pdf-1', 'ser-2', 'ser-3', 'sro-1', 'syg-1', '

None
None
empty VA05

VA0
None
None
empty VA0

VA
['acc-4', 'ace-2', 'acr-12', 'acr-14', 'acr-15', 'acr-2', 'avr-15', 'cha-1', 'cho-1', 'ckr-2', 'cnd-1', 'dbl-1', 'del-1', 'dgk-1', 'dop-1', 'dop-3', 'eat-6', 'gar-2', 'gar-3', 'gbb-1', 'goa-1', 'inx-12', 'inx-3', 'kvs-1', 'mbl-1', 'nca-1', 'nlg-1', 'nlp-21', 'pag-3', 'pkc-1', 'rab-3', 'ric-7', 'rig-3', 'slo-2', 'syg-1', 'tig-2', 'trp-1', 'tsp-7', 'twk-13', 'twk-40', 'twk-43', 'twk-7', 'unc-17', 'unc-3', 'unc-31', 'unc-37', 'unc-4', 'unc-40', 'unc-57', 'unc-6', 'unc-7', 'unc-8', 'unc-9', 'vab-3']
None

VA06
None
None
empty VA06

VA0
None
None
empty VA0

VA
['acc-4', 'ace-2', 'acr-12', 'acr-14', 'acr-15', 'acr-2', 'avr-15', 'cha-1', 'cho-1', 'ckr-2', 'cnd-1', 'dbl-1', 'del-1', 'dgk-1', 'dop-1', 'dop-3', 'eat-6', 'gar-2', 'gar-3', 'gbb-1', 'goa-1', 'inx-12', 'inx-3', 'kvs-1', 'mbl-1', 'nca-1', 'nlg-1', 'nlp-21', 'pag-3', 'pkc-1', 'rab-3', 'ric-7', 'rig-3', 'slo-2', 'syg-1', 'tig-2', 'trp-1', 'tsp-7', 'twk-13', 'twk-40', 'twk-43', 'twk-7', 

None
None
empty VB08

VB0
None
None
empty VB0

VB
['acc-4', 'ace-2', 'acr-12', 'acr-14', 'acr-15', 'acr-2', 'acr-5', 'ceh-12', 'cha-1', 'cho-1', 'ckr-2', 'cnd-1', 'dbl-1', 'del-1', 'dgk-1', 'dop-1', 'dop-3', 'eat-6', 'gar-2', 'gar-3', 'gbb-1', 'glr-4', 'goa-1', 'inx-12', 'inx-3', 'kvs-1', 'nca-1', 'nlg-1', 'nlp-21', 'pag-3', 'pkc-1', 'rab-3', 'ric-7', 'slo-2', 'syg-1', 'tig-2', 'trp-1', 'tsp-7', 'twk-13', 'twk-40', 'twk-43', 'twk-7', 'unc-17', 'unc-3', 'unc-31', 'unc-40', 'unc-57', 'unc-6', 'unc-63', 'unc-7', 'unc-8', 'unc-9']
['ceh-12']

VB09
None
None
empty VB09

VB0
None
None
empty VB0

VB
['acc-4', 'ace-2', 'acr-12', 'acr-14', 'acr-15', 'acr-2', 'acr-5', 'ceh-12', 'cha-1', 'cho-1', 'ckr-2', 'cnd-1', 'dbl-1', 'del-1', 'dgk-1', 'dop-1', 'dop-3', 'eat-6', 'gar-2', 'gar-3', 'gbb-1', 'glr-4', 'goa-1', 'inx-12', 'inx-3', 'kvs-1', 'nca-1', 'nlg-1', 'nlp-21', 'pag-3', 'pkc-1', 'rab-3', 'ric-7', 'slo-2', 'syg-1', 'tig-2', 'trp-1', 'tsp-7', 'twk-13', 'twk-40', 'twk-43', 'twk-7', 'unc-17', 'u

In [10]:
print(NeuronPromoters)

{'ADA': {'GeneExp': ['eat-4', 'flp-8', 'inx-4', 'nsy-5', 'pdf-1', 'unc-86', 'unc-9'], 'UniqExp': None}, 'ADE': {'GeneExp': ['acr-16', 'ast-1', 'bas-1', 'cat-1', 'cat-2', 'cat-4', 'che-2', 'che-3', 'dat-1', 'dop-2', 'dyf-11', 'exp-1', 'gpa-14', 'inx-4', 'inx-7', 'osm-6', 'pdf-1', 'snf-3', 'tax-2', 'trp-4', 'unc-9'], 'UniqExp': None}, 'ADF': {'GeneExp': ['bas-1', 'cat-1', 'che-12', 'che-2', 'che-3', 'dyf-11', 'eat-4', 'egl-30', 'flp-21', 'glb-5', 'gpa-10', 'gpa-13', 'gpa-3', 'ins-1', 'ins-7', 'inx-4', 'kvs-1', 'lin-11', 'mgl-3', 'mps-1', 'ncs-1', 'nlp-3', 'npr-5', 'nsy-5', 'ntr-1', 'ntr-2', 'odr-3', 'odr-4', 'osm-3', 'osm-6', 'osm-9', 'ser-2', 'srb-6', 'srd-1', 'syg-1', 'tax-6', 'tmc-1', 'tph-1'], 'UniqExp': []}, 'ADL': {'GeneExp': ['che-12', 'che-2', 'che-3', 'dyf-11', 'eat-4', 'flp-21', 'flp-4', 'gcy-21', 'gpa-1', 'gpa-11', 'gpa-15', 'gpa-3', 'gpa-5', 'gpa-6', 'gpc-1', 'inx-18', 'kvs-1', 'lin-11', 'mbr-1', 'ncs-1', 'ncx-6', 'ncx-7', 'nlp-10', 'nlp-7', 'nlp-8', 'nsy-5', 'ntr-1', 'ntr-2'

In [11]:
print(len(NeuronPromoters.keys()))

102


In [12]:
print(NeuronPromoters['URA'])

{'GeneExp': ['cfi-1', 'cho-1', 'flp-21', 'flp-22', 'glr-4', 'glr-5', 'sox-2', 'unc-17', 'unc-86', 'unc-9', 'wrk-1'], 'UniqExp': None}


In [13]:
FinalNeuPro = []
for neu in NeuronPromoters.keys():
    NeuPro = NeuronPromoters[neu]['GeneExp']
    NeuPro.insert(0,neu)  
    FinalNeuPro.append(NeuPro)

In [14]:
print(FinalNeuPro)

[['ADA', 'eat-4', 'flp-8', 'inx-4', 'nsy-5', 'pdf-1', 'unc-86', 'unc-9'], ['ADE', 'acr-16', 'ast-1', 'bas-1', 'cat-1', 'cat-2', 'cat-4', 'che-2', 'che-3', 'dat-1', 'dop-2', 'dyf-11', 'exp-1', 'gpa-14', 'inx-4', 'inx-7', 'osm-6', 'pdf-1', 'snf-3', 'tax-2', 'trp-4', 'unc-9'], ['ADF', 'bas-1', 'cat-1', 'che-12', 'che-2', 'che-3', 'dyf-11', 'eat-4', 'egl-30', 'flp-21', 'glb-5', 'gpa-10', 'gpa-13', 'gpa-3', 'ins-1', 'ins-7', 'inx-4', 'kvs-1', 'lin-11', 'mgl-3', 'mps-1', 'ncs-1', 'nlp-3', 'npr-5', 'nsy-5', 'ntr-1', 'ntr-2', 'odr-3', 'odr-4', 'osm-3', 'osm-6', 'osm-9', 'ser-2', 'srb-6', 'srd-1', 'syg-1', 'tax-6', 'tmc-1', 'tph-1'], ['ADL', 'che-12', 'che-2', 'che-3', 'dyf-11', 'eat-4', 'flp-21', 'flp-4', 'gcy-21', 'gpa-1', 'gpa-11', 'gpa-15', 'gpa-3', 'gpa-5', 'gpa-6', 'gpc-1', 'inx-18', 'kvs-1', 'lin-11', 'mbr-1', 'ncs-1', 'ncx-6', 'ncx-7', 'nlp-10', 'nlp-7', 'nlp-8', 'nsy-5', 'ntr-1', 'ntr-2', 'ocr-1', 'ocr-2', 'odr-4', 'osm-3', 'osm-6', 'osm-9', 'qui-1', 'rgs-3', 'sod-1', 'srb-6', 'sre-1',

In [20]:
csvfile = open('/Users/apple/Desktop/IGEM/post/neuron network/FullPromToNeu.csv','w',newline = '')
csvfile_write = csv.writer(csvfile)

for neu in FinalNeuPro:
    print(neu)
    csvfile_write.writerow(neu)
csvfile.close()

['ADA', 'eat-4', 'flp-8', 'inx-4', 'nsy-5', 'pdf-1', 'unc-86', 'unc-9']
['ADE', 'acr-16', 'ast-1', 'bas-1', 'cat-1', 'cat-2', 'cat-4', 'che-2', 'che-3', 'dat-1', 'dop-2', 'dyf-11', 'exp-1', 'gpa-14', 'inx-4', 'inx-7', 'osm-6', 'pdf-1', 'snf-3', 'tax-2', 'trp-4', 'unc-9']
['ADF', 'bas-1', 'cat-1', 'che-12', 'che-2', 'che-3', 'dyf-11', 'eat-4', 'egl-30', 'flp-21', 'glb-5', 'gpa-10', 'gpa-13', 'gpa-3', 'ins-1', 'ins-7', 'inx-4', 'kvs-1', 'lin-11', 'mgl-3', 'mps-1', 'ncs-1', 'nlp-3', 'npr-5', 'nsy-5', 'ntr-1', 'ntr-2', 'odr-3', 'odr-4', 'osm-3', 'osm-6', 'osm-9', 'ser-2', 'srb-6', 'srd-1', 'syg-1', 'tax-6', 'tmc-1', 'tph-1']
['ADL', 'che-12', 'che-2', 'che-3', 'dyf-11', 'eat-4', 'flp-21', 'flp-4', 'gcy-21', 'gpa-1', 'gpa-11', 'gpa-15', 'gpa-3', 'gpa-5', 'gpa-6', 'gpc-1', 'inx-18', 'kvs-1', 'lin-11', 'mbr-1', 'ncs-1', 'ncx-6', 'ncx-7', 'nlp-10', 'nlp-7', 'nlp-8', 'nsy-5', 'ntr-1', 'ntr-2', 'ocr-1', 'ocr-2', 'odr-4', 'osm-3', 'osm-6', 'osm-9', 'qui-1', 'rgs-3', 'sod-1', 'srb-6', 'sre-1', 'sr

In [17]:
print(len(FinalNeuPro))

102
